# Introductin: Business Problem

I live in Dhaka,Bangladesh. Dhaka is the capital and the lagrest city of Bangladesh. As well as it is the ninth-largest and the
sixth-most densely population city in the world, as of 2011. In addition, it is the financial, commercial capital of Bangladesh.

In this project, I will try to recomend to  potential investor or stackholders or business owner to open a optimal business as 
possible as in the Dhaka city.

# Data

My business recomendation based on foursquare location data. So, At fast I wil extract geographical coordinates of Dhaka using 
python package and then create a foursquare url using my foursquare client_id, client_secret..etc.Fianlly, extract the required
data as json format.


In [1]:
# Get longitude & lattitude using Dhaka's location 
from geopy.geocoders import Nominatim
Address='Dhaka,Bangladesh'
geolocator=Nominatim(user_agent='Dhaka explorer')
location=geolocator.geocode(Address)
longitude=location.longitude
latitude=location.latitude
print('The Geographical coordinates of Dhaka is {},{}'.format(longitude,latitude))

The Geographical coordinates of Dhaka is 90.4126466,23.810651


In [2]:
# foursquare api credential
CLIENT_ID='PVFHMYLB5SNS10OVXLYOKRKT0YFJWLCZUSUH0CARQO4QMB3N'
CLIENT_SECRET='TROKPKIXHV2EQJUDC32KDANASYMXNZ1MZMLHC30H5TWU3SKL'
VERSION='20210531'
LIMIT=100
print('foursquare CLIENT_ID & CLIENT_SECRET are {},{}'.format(CLIENT_ID,CLIENT_SECRET))

foursquare CLIENT_ID & CLIENT_SECRET are PVFHMYLB5SNS10OVXLYOKRKT0YFJWLCZUSUH0CARQO4QMB3N,TROKPKIXHV2EQJUDC32KDANASYMXNZ1MZMLHC30H5TWU3SKL


In [3]:
# build url to extract required data from foursquare.com
latitude=23.810651
longitude=90.4126466
radius=10000
LIMIT=100
url='http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

In [4]:
# import required libraries 
import requests
import json
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e3a7d9aa081c3491caead2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Dhaka',
  'headerFullLocation': 'Dhaka',
  'headerLocationGranularity': 'city',
  'totalResults': 177,
  'suggestedBounds': {'ne': {'lat': 23.90065109000009,
    'lng': 90.51083608324078},
   'sw': {'lat': 23.72065090999991, 'lng': 90.31445711675923}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '567bdc18498e1d6a154f6648',
       'name': 'Burger Lab',
       'location': {'lat': 23.812615,
        'lng': 90.424544,
        'labeledLatLngs': [{'label': 'display',
          'lat': 23.812615,
          'lng': 90.424544}],
        'distance': 1231,
        'cc': 'BD',
    

# Methodology

After extracting required data from foursquare as json format. Now, we have to simplify the structue of data and we have get
a simple format of that data for our analysis make easy. For simplification attempt, At fast we will brow a function that was
created in foursquare lab to extract the category of the different venue in the Dhaka. code are following

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

And Now we will extract required features such as venue_name, venue_category, venue_crossStreet, venue distance form the center 
point of dhaka city, venue longitude, venue latitude

In [6]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address','venue.location.crossStreet','venue.location.distance','venue.location.lat','venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
# What does apply function in python
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

C:\Users\ASUS\downloder\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,address,crossStreet,distance,lat,lng
0,Burger Lab,Burger Joint,NaN,NaN,1231,23.812615,90.424544
1,Tabaq,Coffee Shop,Jamuna Future Park,5th Floor,1172,23.812956,90.423879
2,Movenpick,Ice Cream Shop,Gulshan avenue,NaN,2057,23.792333,90.415373
3,Bitter Sweet,Café,gulshan,NaN,1684,23.795532,90.413268
4,Bistro E,Bistro,NE 12 North Ave.,NaN,1695,23.796046,90.417373
...,...,...,...,...,...,...,...
95,"Western Grill, Mirpur",Fast Food Restaurant,"149/Ga/1 Shah Ali Bagh, Mirpur -2",Near Janata Housing,5703,23.801170,90.357614
96,Almas Super Store,Department Store,"Road No.: 8/A, Dhanmondi",NaN,8430,23.744308,90.372741
97,Cilantro,Café,"49, Shatmasjid Road,",Dhanmondi,8309,23.746544,90.370859
98,d'Popeyes & Miss Vanellis,Fast Food Restaurant,"91, Satmasjid Road,",Dhanmondi 12/A,8249,23.747341,90.370562


In [7]:
# Rename Column name
nearby_venues=nearby_venues.rename(columns={'distance':'distance_from_center','lat':'venue_lat','lng':'venue_lng'})
nearby_venues

,name,categories,address,crossStreet,distance_from_center,venue_lat,venue_lng
0,Burger Lab,Burger Joint,NaN,NaN,1231,23.812615,90.424544
1,Tabaq,Coffee Shop,Jamuna Future Park,5th Floor,1172,23.812956,90.423879
2,Movenpick,Ice Cream Shop,Gulshan avenue,NaN,2057,23.792333,90.415373
3,Bitter Sweet,Café,gulshan,NaN,1684,23.795532,90.413268
4,Bistro E,Bistro,NE 12 North Ave.,NaN,1695,23.796046,90.417373
...,...,...,...,...,...,...,...
95,"Western Grill, Mirpur",Fast Food Restaurant,"149/Ga/1 Shah Ali Bagh, Mirpur -2",Near Janata Housing,5703,23.801170,90.357614
96,Almas Super Store,Department Store,"Road No.: 8/A, Dhanmondi",NaN,8430,23.744308,90.372741
97,Cilantro,Café,"49, Shatmasjid Road,",Dhanmondi,8309,23.746544,90.370859
98,d'Popeyes & Miss Vanellis,Fast Food Restaurant,"91, Satmasjid Road,",Dhanmondi 12/A,8249,23.747341,90.370562


# Analysis

Now, The data is extracted. So, it is the time to analyze the data. Remember, 
Before analysis, We have to check is there or how many null values exists in our dataframe.

In [8]:
# check all null values
nearby_venues.isnull().sum()

name                     0
categories               0
address                 17
crossStreet             58
distance_from_center     0
venue_lat                0
venue_lng                0
dtype: int64

We have seen lot of null values exist in our data. Also, we seen that all null values in address, crossStreet and distance 
cloumn. Since, Null values affect significantly in the whole analysis and we ensure that these features are not important
for our analysis. So, Droped those cloumn contain null values

In [9]:
# drop the columnn these contain more null value and extract the dataframe that contain only 5 columns these are required for our analysis
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.distance','venue.location.lat','venue.location.lng']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns]

# filter the category for each row
# What does apply function in python
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

nearby_venues1

C:\Users\ASUS\downloder\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,distance,lat,lng
0,Burger Lab,Burger Joint,1231,23.812615,90.424544
1,Tabaq,Coffee Shop,1172,23.812956,90.423879
2,Movenpick,Ice Cream Shop,2057,23.792333,90.415373
3,Bitter Sweet,Café,1684,23.795532,90.413268
4,Bistro E,Bistro,1695,23.796046,90.417373
...,...,...,...,...,...
95,"Western Grill, Mirpur",Fast Food Restaurant,5703,23.801170,90.357614
96,Almas Super Store,Department Store,8430,23.744308,90.372741
97,Cilantro,Café,8309,23.746544,90.370859
98,d'Popeyes & Miss Vanellis,Fast Food Restaurant,8249,23.747341,90.370562


In [10]:
# Rename the column names
nearby_venues1=nearby_venues1.rename(columns={'distance':'distance_from_center','lat':'venue_lat','lng':'venue_lng','name':'venue_name','categories':'venue_categories'})
nearby_venues1

,venue_name,venue_categories,distance_from_center,venue_lat,venue_lng
0,Burger Lab,Burger Joint,1231,23.812615,90.424544
1,Tabaq,Coffee Shop,1172,23.812956,90.423879
2,Movenpick,Ice Cream Shop,2057,23.792333,90.415373
3,Bitter Sweet,Café,1684,23.795532,90.413268
4,Bistro E,Bistro,1695,23.796046,90.417373
...,...,...,...,...,...
95,"Western Grill, Mirpur",Fast Food Restaurant,5703,23.801170,90.357614
96,Almas Super Store,Department Store,8430,23.744308,90.372741
97,Cilantro,Café,8309,23.746544,90.370859
98,d'Popeyes & Miss Vanellis,Fast Food Restaurant,8249,23.747341,90.370562


In [11]:
# Check all the null values
nearby_venues1.isnull().sum()

venue_name              0
venue_categories        0
distance_from_center    0
venue_lat               0
venue_lng               0
dtype: int64

In [12]:
import folium
dk_map=folium.Map(location=[latitude,longitude],zoom_start=12)
for lat, lng,name,category in zip(nearby_venues1['venue_lat'],nearby_venues1['venue_lng'],nearby_venues1['venue_name'],nearby_venues1['venue_categories']):
        label='{},{}'.format(name,category)
        label=folium.Popup(label,parse_html=True)
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            popup=label,
            color='red',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(dk_map)
        

In [13]:
dk_map 

In [16]:
# Group the dataframe based on venue catagories feature and count them
nearby_venues1_grp=nearby_venues1.groupby('venue_categories').count()
nearby_venues1_grp.reset_index(inplace=True)

In [17]:
# sort the final dataframe descending order based on Num_of_venue_each_categories
nearby_venues1_grp_o=nearby_venues1_grp.sort_values(by=['venue_name'],ascending=False)
nearby_venues1_grp_o.rename(columns={'venue_name':'Num_Of_venue_each_catagories'})

,venue_categories,Num_Of_venue_each_catagories,distance_from_center,venue_lat,venue_lng
11,Café,11,11,11,11
13,Coffee Shop,8,8,8,8
37,Restaurant,7,7,7,7
31,Nightclub,4,4,4,4
2,Asian Restaurant,4,4,4,4
33,Pizza Place,4,4,4,4
19,Fast Food Restaurant,4,4,4,4
12,Clothing Store,4,4,4,4
1,Arts & Crafts Store,3,3,3,3
24,Ice Cream Shop,3,3,3,3


# Results and Discussion

From final dataframe, we clearly seen food business is popular in Dhaka city. Most visited venues are food venues in order cafe,normal restaurant, coffe shop etc . explicitly restaurant bussiness in more popular in Dhaka city because we seen 7 normal 
restaurant as well as differnt country's special restaurant such as Asian, Indian, Italian, Mexican, Portugeese and so on...
as well as The Night Club is also popular in Dhaka City. 
So, I will recomend investor or stackholder to open a restaurant(specila type of or others) if he interested in food business
or Night club . Cause We seen that these two types of business retrun more value of investment.


# Conclusion

Purpose of this project was recomendation to investor or stackholder to start a business in Dhaka city using foursquare data
and required analysis.So, I have tried  my best to find optimal business through data analysis to potential investor who has 
interested to start a business in Dhaka and I hope, Potential investor would get a direction to choose a right business which 
more return their investment and eventually they will be benefited.